출처:https://www.kaggle.com/code/ashishpatel26/smote-with-model-lightgbm

### lightgbm(Light Gradient Boosting Machine):
 기계 학습을위한 무료 오픈 소스 분산 경사하강법(Gradient Boostiong) 프레임 워크
### tqdm
 실행 시간이 긴 코드에 대한 진행상황을 확인 가능하도록 하는 라이브러리
### dask
 병렬컴퓨팅을 위한 라이브러리

### dask.dataframe을 import 할 때 오류가 발생할 경우 아래 코드 실행
```
pip install dask[dataframe] --upgrade
```

In [2]:
pip install dask[dataframe] --upgrade

     |████████████████████████████████| 136 kB 4.2 MB/s 


In [ ]:
import warnings
import itertools
import numpy as np 
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, log_loss
import dask
import dask.dataframe as dd
import gc

from yellowbrick.text import TSNEVisualizer

%matplotlib inline
plt.style.use("fivethirtyeight")

import os
print(os.listdir('/content/drive/MyDrive/Colab Notebooks/data_analysis/reducing-commercial-aviation-fatalities'))

warnings.simplefilter(action='ignore')
sns.set_style('whitegrid')

데이터 불러오기

In [ ]:
def gc():
    print(gc.collect())


dtypes = {"crew": "int8",
          "experiment": "category",
          "time": "float32",
          "seat": "int8",
          "eeg_fp1": "float32",
          "eeg_f7": "float32",
          "eeg_f8": "float32",
          "eeg_t4": "float32",
          "eeg_t6": "float32",
          "eeg_t5": "float32",
          "eeg_t3": "float32",
          "eeg_fp2": "float32",
          "eeg_o1": "float32",
          "eeg_p3": "float32",
          "eeg_pz": "float32",
          "eeg_f3": "float32",
          "eeg_fz": "float32",
          "eeg_f4": "float32",
          "eeg_c4": "float32",
          "eeg_p4": "float32",
          "eeg_poz": "float32",
          "eeg_c3": "float32",
          "eeg_cz": "float32",
          "eeg_o2": "float32",
          "ecg": "float32",
          "r": "float32",
          "gsr": "float32",
          "event": "category",
         }

def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else: 
        print('Confusion matrix, without normalization')
    
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max()/2
    for i, j in itertools,product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i,format(cm[i,j],fmt),
                 gorizontalalignment="center",
                 color='white' if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
%%time
train = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_analysis/reducing-commercial-aviation-fatalities/train.csv',
                    blocksize = 256e6, dtype=dtypes)

In [ ]:
%%time
test = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_analysis/reducing-commercial-aviation-fatalities/test.csv',
                    blocksize = 256e6, dtype=dtypes)
# test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_analysis/reducing-commercial-aviation-fatalities/test.csv',dtype=dtypes)

In [ ]:
%%time
train = train.compute()
print("Training shape : ", train.shape)

In [ ]:
%%time
test = test.compute()
print("Testing shape : ", test.shape)
# print(gc.collect())

In [ ]:
train_df = train.copy()
test_df = test.copy()

In [ ]:
import gc
gc.collect()

In [ ]:
test_id = test_df['id']

dic = {'A': 0,'B':1,'C': 2, 'D':3}
dic1 = {'CA':0, 'DA':1, 'SS':3, 'LOFT':4}
train_df['event'] = train_df['event'].apply(lambda x:dic[x])
train_df['event'] = train_df['event'].astype('int8')
train_df['experiment'] = train_df['experiment'].apply(lambda x:dic1[x])
test_df['experiment'] = test_df['experiment'].apply(lambda x:dic1[x])

train_df['experiment'] = train_df['experiment'].astype('int8')
test_df['exdperiment'] = test_df['experiment'].astype('int8')

y = train_df['event']

In [ ]:
tr_col = train_df.columns

In [ ]:
test_df.head()

In [ ]:
train_df.shape, test.shape

#EDA

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(train_df['event'])
plt.xlabel('State of the pilot', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title("'event' column's Distribution", fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot('experiment', hue='event', data=train_df)
plt.xlabel('Experiment and state of the pilot', fontsize=12)
plt.ylabel('Count(log)', fontsize=12)
plt.yscale('log')
plt.title("'experiment' column's Distribution'", fontsize=15)
plt.show()

### Seat

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot('event', hue='seat', data=train_df)
plt.xlabel('Seat and state of the pilot', fontsize=12)
plt.ylabel('Count(log)', fontsize=12)
plt.yscale('log')
plt.title("'seat' column's Distribution", fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.violinplot(x='event',y='time', data=train_df.sample(50000))
plt.xlabel('Event',fontsize=12)
plt.ylabel('Time(s)',fontsize=12)
plt.title('Distribution of events over time', fontsize= 15)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.distplot(test_df['time'], label='Test set')
sns.distplot(train_df['time'], label='Train set')
plt.xlabel('Time(s)', fontsize=12)
plt.legend()
plt.title('Reparition of the time feature', fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.histplot(test_df['time'], label='Test set', color='r')
sns.histplot(train_df['time'], label='Train set',color='b')
plt.xlabel('Time(s)', fontsize=12)
plt.legend()
plt.title('Distribution of time values', fontsize=15)
plt.show()

### EGG

In [ ]:
eeg_features = ["eeg_fp1", "eeg_f7", "eeg_f8", "eeg_t4", "eeg_t6", "eeg_t5", "eeg_t3", "eeg_fp2", "eeg_o1", "eeg_p3", "eeg_pz", "eeg_f3", "eeg_fz", "eeg_f4", "eeg_c4", "eeg_p4", "eeg_poz", "eeg_c3", "eeg_cz", "eeg_o2"]

In [ ]:
plt.figure(figsize=(15,10))
i = 0

for egg in eeg_features:
    i += 1
    plt.subplot(5,4,i)
    sns.boxplot(x='event', y=egg, data=train_df.sample(50000), showfliers=False)
plt.show()

In [ ]:
plt.figure(figsize=(20,25))
plt.title('Eeg features distributions')
i = 0

for eeg in eeg_features:
    i+=1
    plt.subplot(5,4,i)
    sns.distplot(test_df.sample(10000)[eeg], label='Test set', hist =False)
    sns.distplot(train_df.sample(10000)[eeg], label='Train set', hist=False)
    plt.xlim((-500, 500))
    plt.legend()
    plt.xlabel(eeg, fontsize=12)

plt.show()

###Electrocardiogram

In [ ]:
plt.figure(figsize=(15,10))
sns.violinplot(x='event',y='ecg', data=train_df.sample(50000))
plt.ylabel('Electrocardiogram Signal(mv)', fontsize=12)
plt.xlabel('Event', fontsize=12)
plt.title('Electrocardiogram signal influence', fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.distplot(test_df['ecg'],label='Test set')
sns.distplot(train_df['ecg'],label='Train set')
plt.legend()
plt.xlabel('Electrocardiogram Signal(mv)', fontsize=12)
plt.title('Electrocardiogram Signal Distribution', fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.violinplot(x='event', y='r', data=train_df.sample(50000))
plt.ylabel('Resiration Signal(mv)',fontsize=12)
plt.xlabel('Event', fontsize=12)
plt.title('Respiration influence', fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.distplot(test_df['r'],label = 'Test set')
sns.distplot(train_df['r'],label = 'Train set')
plt.legend()
plt.xlabel('Respiration Signal (mv)', fontsize = 12)
plt.ylabel('Respiration Signal Distribution', fontsize = 12)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.violinplot('event','gsr', data=train_df.sample(50000))
plt.ylabel('Electrodermal activity measure (mv)', fontsize=12)
plt.xlabel('Event', fontsize=12)
plt.title('Electrodermal activity influence', fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize =(15,10))
sns.distplot(test_df['gsr'], label='Test set')
sns.distplot(train_df['gsr'], label='Train set')
plt.legend()
plt.xlabel('Electrodermal activity measure(mv)', fontsize = 12)
plt.title('Electrodermal activity Distribution', fontsize =15)
plt.show()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
feature_n = ["eeg_fp1", "eeg_f7", "eeg_f8", "eeg_t4", "eeg_t6", "eeg_t5", "eeg_t3", "eeg_fp2", "eeg_o1", "eeg_p3", "eeg_pz", "eeg_f3", "eeg_fz", "eeg_f4", "eeg_c4", "eeg_p4", "eeg_poz", "eeg_c3", "eeg_cz", "eeg_o2", "ecg", "r", "gsr"]

In [ ]:
train_df['pilot'] = 100 * train_df['seat'] + train_df['crew']
test_df['pilot'] = 100 * test_df['seat'] + test_df['crew']

print('Number of pilots : ', len(train_df['pilot'].unique()))

In [ ]:
train_df.shape, test_df.shape

In [ ]:
tr_col = train_df.columns

In [ ]:
scaler = MinMaxScaler()
train_df = scaler.fit_transform(train_df)
train_df = pd.DataFrame(train_df)
train_df.columns = tr_col
describe(train_df)

## Solve Class Imvalance Problem Using SMOTE
###SMOTE(synthetic minority oversampling technique)
 데이터의 수가 적은 클래스의 표본을 가져와 임의의 값을 추가하여 새로운 샘플을 만들어 데이터에 추가하는 오버샘플링 방식이다.
 데이터의 클래스 별 분포가 불균형할 때 사용하여 균형을 맞춘다.

In [ ]:
#train_df = normalize_by_pilots(train_df)
#test_df = normalize_by_pilots(test_df)
from imblearn.over_sampling import SMOTE
train_df, y = SMOTE().fit_resample(train_df, y.ravel())

In [ ]:
train_df.shape

In [ ]:
train_df = pd.DataFrame(train_df)
train_df.columns = tr_col
train_df.head()

### train 데이터와 test 데이터 나누기




In [ ]:
print(train_df.shape, test.shape)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=32)
print(f"Training on {train_df.shape[0]} samples.")

In [ ]:
features = ['crew','seat'] + features_n

def run_lgb(df_train, df_test):
    dic = {'A':0,'B':1,'C':2,'D':3}
    try:
        df_train['event'] = df_train["event"].apply(lambda x: dic[x])
        df_test['event'] = df_test['event'].apply(lambda x: dic[x])
    except:
        pass
    
    params = {'objective':'multiclass',
              'num_class': 4,
              'metric' : 'multi_error',
              'num_leaves' : 30,
              'min_child_weight' : 50,
              'learning_rate' : 0.1,
              'bagging_fraction' : 0.7,
              'reg_alpha' : 0.15,
              'reg_lambda' : 0.15,
              'min_child_weight' : 50,
              'bagging_seed' : 420,
              'verbosity' : -1}
    
    lg_train = lgb.Dataset(df_train[features], label=(df_train['event']))
    lg_test = lgb.Dataset(df_test[features], label=(df_test['event']))
    model = lgb.train(params, lg_train, 1000, valid_sets=[lg_test], early_stopping_rounds=50,verbose_eval=100)

    return model

In [ ]:
model = run_lgb(train_df, val_df)

### Feature importance